In [ ]:
import numpy as np

train_input = np.load('train_input.npy')
train_labels = np.load('train_labels.npy')
val_input = np.load('val_input.npy')
val_labels = np.load('val_labels.npy')
test_input = np.load('test_input.npy')
test_labels = np.load('test_labels.npy')

print(f"Train input shape: {train_input.shape}, Train labels shape: {train_labels.shape}")
print(f"Validation input shape: {val_input.shape}, Validation labels shape: {val_labels.shape}")
print(f"Test input shape: {test_input.shape}, Test labels shape: {test_labels.shape}")